In [135]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [136]:


import numpy as np
import pandas as pd

import olympus
from olympus.objects import ParameterVector
from olympus.datasets import Dataset
from olympus.campaigns import ParameterSpace, Campaign
from olympus.surfaces import Surface
from olympus.objects import (
    ParameterContinuous, 
    ParameterDiscrete,
    ParameterCategorical,
)

from chimera import Chimera

In [107]:
# returns list of lists
def fonseca(params): 
    ''' 2 parameters, 2 objectives, minimize '''
    params = np.array(params)  
#   vector = np.array([params[key_name]['samples'][0] for key_name in ['x', 'y']])
    obj_0  = 1 - np.exp( - np.sum((params - 1. / np.sqrt(len(params)))**2))
    obj_1  = 1 - np.exp( - np.sum((params + 1. / np.sqrt(len(params)))**2))
#     params['obj_0'] = obj_0
#     params['obj_1'] = obj_1
    return [[obj_0, obj_1]]


def viennet(params):
    ''' 2 parameters, 3 objectives, minimize '''
    params = np.array(params)
    obj_0 = 0.5*(params[0]**2 + params[1]**2) + np.sin(params[0]**2 + params[1]**2)
    obj_1 = (((3*params[0]-2*params[1]+4)**2)/8) + (((params[0]-params[1]+1)**2) / 27) + 15
    obj_2 = (1/(params[0]**2 + params[1]**2 + 1)) - (1.1 * np.exp(-(params[0]**2 + params[1]**2)))
    
    return [[obj_0, obj_1, obj_2]]

def zdt1(params):
    ''' Zitzler–Deb–Thiele's function N. 1 
    2-30 parameters, 2 objectives, minimize 
    '''
    params = np.array(params)
    obj_0 = params[0]
    g = 1 + ((9/29)*np.sum(params))
    h = 1 - np.sqrt(obj_0/g)
    obj_1 = g*h
    
    return [[obj_0, obj_1]]


def zdt2(params):
    ''' Zitzler–Deb–Thiele's function N. 2
    2-30 parameters, 2 objectives, minimize '''
    params = np.array(params)
    obj_0 = params[0]
    g = 1 + ((9/29)*np.sum(params))
    h = 1 - (obj_0/g)**2
    obj_1 = g*h
    
    return [[obj_0, obj_1]]
    
    
def zdt3(params):
    ''' Zitzler–Deb–Thiele's function N. 2
    2-30 parameters, 2 objectives, minimize '''
    params = np.array(params)
    obj_0 = params[0]
    g = 1 + ((9/29)*np.sum(params))
    h = 1 - np.sqrt(obj_0/g) - (obj_0/g)*np.sin(10*np.pi*obj_0)
    obj_1 = g*h
    
    return [[obj_0, obj_1]]

In [108]:
param_space = ParameterSpace()

param_space.add(
    ParameterContinuous(
        name='param_0',
        low=0., 
        high=1.,
        
    )
)

param_space.add(
    ParameterContinuous(
        name='param_1',
        low=0., 
        high=1.,
        
    )
)

print(param_space)

Continuous (name='param_0', low=0.0, high=1.0, is_periodic=False)
Continuous (name='param_1', low=0.0, high=1.0, is_periodic=False)


In [109]:
test_param = [0.01, 0.99]
test_param_3 = [0.01, 0.99, 0.67]
test_param_30 = [np.random.uniform() for _ in range(30)]
test_param_40 = [np.random.uniform() for _ in range(40)]
test_param_1 = [0.6]

In [110]:
print('fonseca : ', fonseca(test_param))
print('viennet :', viennet(test_param))
print('ZDT 1 : ', zdt1(test_param))
print('ZDT 2 : ', zdt2(test_param))
print('ZDT 3 : ', zdt3(test_param))

fonseca :  [[0.432200228643724, 0.9664397706162793]]
viennet : [[1.3207087583906336, 15.525327314814815, 0.09223984644941902]]
ZDT 1 :  [[0.01, 1.195874533291739]]
ZDT 2 :  [[0.01, 1.3102685117967332]]
ZDT 3 :  [[0.01, 1.1927843633479895]]


In [111]:
surface = Surface(kind='Dejong')
surface.run(test_param)

[[4.427188724235731]]

In [112]:
surface = Surface(kind='MultFonseca')
surface.run(test_param)

[WARNING] Surface MultFonseca is only defined with 2 objectives: setting `value_dim`=2


[[[0.432200228643724, 0.9664397706162793]]]

In [113]:
surface = Surface(kind='MultViennet')
surface.run(test_param)

[WARNING] Surface MultViennet is only defined with 3 objectives: setting `value_dim`=3


[[[1.3207087583906336, 15.525327314814815, 0.09223984644941902]]]

In [114]:
surface = Surface(kind='MultZdt1', param_dim=30)
surface.run(test_param_30)

[WARNING] Surface MultZdt1 is only defined with 2 objectives: setting `value_dim`=2


[[[0.1928607048702562, 4.496591542997461]]]

In [115]:
surface = Surface(kind='MultZdt1')
surface.run(test_param)

[WARNING] Surface MultZdt1 is only defined with 2 objectives: setting `value_dim`=2


[[[0.01, 1.195874533291739]]]

In [137]:
#olympus.surfaces.get_surfaces_list()

In [170]:
surface = Surface(kind='MultFonseca')
print(surface)

campaign = Campaign()
campaign.set_param_space(surface.param_space)
campaign.set_value_space(surface.value_space)


--> noise:     None
--> param_dim: 2
--> value_dim: 2


[WARNING] Surface MultFonseca is only defined with 2 objectives: setting `value_dim`=2


In [171]:
print(campaign)


--> accepts:          param_vector
--> dataset_kind:     n/a
--> emulator_type:    n/a
--> goal:             minimize
--> id:               97823643
--> measurement_name: n/a
--> model_kind:       n/a
--> observations:     <olympus.campaigns.observations.Observations object at 0x7fb879634b10>
--> param_space:      Continuous (name='param_0', low=0.0, high=1.0, is_periodic=False)
Continuous (name='param_1', low=0.0, high=1.0, is_periodic=False)
--> planner_kind:     n/a
--> surface_kind:     n/a
--> value_space:      Continuous (name='value_0', low=0.0, high=1.0, is_periodic=False)
Continuous (name='value_1', low=0.0, high=1.0, is_periodic=False)


In [176]:
samples = ParameterVector().from_array([0.01, 0.99], param_space=surface.param_space)
print('samples : ', samples)
measurements = surface.run(samples.to_array())[0][0]

print('measurements : ', measurements)


campaign.add_observation([samples], measurements)


samples :  ParamVector(param_0 = 0.01, param_1 = 0.99)
measurements :  [0.432200228643724, 0.9664397706162793]
VALUE :  [0.432200228643724, 0.9664397706162793]
PARAM :  [ParamVector(param_0 = 0.01, param_1 = 0.99)]
param :  ParamVector(param_0 = 0.01, param_1 = 0.99)


In [177]:
campaign.observations.get_values().shape

(6, 2)

In [ ]:
# ParamVector(param_0 = 0.01, param_1 = 0.99)
# [[0.432200228643724, 0.9664397706162793]]
# VALUE :  [[0.432200228643724, 0.9664397706162793]]
# PARAM :  [ParamVector(param_0 = 0.01, param_1 = 0.99)]
# param :  ParamVector(param_0 = 0.01, param_1 = 0.99)